In [181]:
!pip install -qU geopandas

# Load libraries

In [183]:
import numpy as np
import pandas as pd
import geopandas as gpd
from google.colab import drive

In [187]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Set with the location of the .csv file and the filename.

In [188]:
#DIR = './2013_2022/All Incidents/All_Incidents_2013_2022_DINS/'
#DIR = '/content/drive/MyDrive/RelativeRisk_Analysis/data/'
DIR = '/content/drive/MyDrive/Risk_Analysis_DINS/data/'
filename = 'dins_2017_2022.csv'

In [189]:
df = pd.read_csv(DIR + filename)

# Description

In [190]:
df.head()

,DAMAGE,ROOFCONSTRUCTION,EAVES,VENTSCREEN,EXTERIORSIDING,WINDOWPANE,DECKPORCHONGRADE,DECKPORCHELEVATED,PATIOCOVER,FENCE,YEARBUILT,LATITUDE,LONGITUDE,distance_meter,DISTANCE
0,Destroyed (>50%),Asphalt,Enclosed,"Mesh Screen <= 1/8""",Combustible,Multi Pane,Composite,No Deck/Porch,Combustible,Combustible,1953,39.773030,-121.578383,52.432312,172.022027
1,Destroyed (>50%),Asphalt,Enclosed,"Mesh Screen <= 1/8""",Combustible,Multi Pane,Composite,No Deck/Porch,Combustible,Non Combustible,1973,39.781616,-121.607243,41.718953,136.873208
2,Destroyed (>50%),Asphalt,Enclosed,"Mesh Screen <= 1/8""",Combustible,Multi Pane,Composite,Wood,No Patio Cover/Carport,Non Combustible,1974,39.780287,-121.585203,36.386770,119.379170
3,Affected (1-9%),Asphalt,Enclosed,"Mesh Screen <= 1/8""",Combustible,Multi Pane,Composite,No Deck/Porch,No Patio Cover/Carport,Non Combustible,1979,39.734655,-121.571064,90.943286,298.370371
4,Affected (1-9%),Asphalt,Enclosed,"Mesh Screen <= 1/8""",Combustible,Multi Pane,Composite,No Deck/Porch,No Patio Cover/Carport,Combustible,1985,39.766168,-121.588601,30.136204,98.872064


# Missing values

For all DINS

In [191]:
pd.DataFrame(df.isna().sum(), columns=['Missing values']).sort_values(by='Missing values', ascending=False)

,Missing values
YEARBUILT,34075
PATIOCOVER,16871
FENCE,16868
DECKPORCHELEVATED,16867
DECKPORCHONGRADE,16866
VENTSCREEN,4465
WINDOWPANE,4425
EAVES,4416
EXTERIORSIDING,4357
ROOFCONSTRUCTION,4340


## Start processing

Backup to avoid load the data again...

In [192]:
df_filter = df.copy() # A copy to work with

In [193]:
df_filter.drop('distance_meter', inplace=True, axis=1)

### Column analysis

#### Damage

Convert values to classes.

In [194]:
df_filter['DAMAGE'].unique()

array(['Destroyed (>50%)', 'Affected (1-9%)', 'Minor (10-25%)',
       'Major (26-50%)', 'No Damage', 'Inaccessible'], dtype=object)

In [195]:
damage_labels = ['Inaccessible', 'No Damage', 'Affected (1-9%)', 'Minor (10-25%)', 'Major (26-50%)', 'Destroyed (>50%)']
damage_values = [np.nan, 0, 1, 1, 1, 2]
df_filter['DAMAGE'].replace(
    damage_labels,
    damage_values,
    inplace=True
)

#### Latitude and Longitude

Nothing to do with them.

In [196]:
df_filter['LATITUDE'].describe()

count    87157.000000
mean        38.508508
std          1.867543
min         32.592548
25%         38.353291
50%         38.771461
75%         39.765584
max         41.991195
Name: LATITUDE, dtype: float64

In [197]:
df_filter['LONGITUDE'].describe()

count    87157.000000
mean      -121.309698
std          1.428654
min       -123.774580
25%       -122.317856
50%       -121.603289
75%       -120.863909
max       -116.444013
Name: LONGITUDE, dtype: float64

#### DECKPORCHONGRADE

Text to categorical values.

In [198]:
df_filter['DECKPORCHONGRADE'].unique()

array(['Composite', 'Masonry/Concrete', 'No Deck/Porch', 'Unknown',
       'Wood', nan], dtype=object)

There are some `' '` values. We will replace them by NaN.


In [199]:
deck_porch_labels = ['Unknown', 'No Deck/Porch', 'Masonry/Concrete', 'Composite', 'Wood', ' ']
deck_porch_values = [np.nan, 0, 1, 2, 3, np.nan]
df_filter['DECKPORCHONGRADE'].replace(
    deck_porch_labels,
    deck_porch_values,
    inplace=True
)

In [200]:
print(df_filter['DECKPORCHONGRADE'].unique())

[ 2.  1.  0. nan  3.]


#### DECKPORCHELEVATED

Text to categorical values.

In [201]:
df_filter['DECKPORCHELEVATED'].unique()

array(['No Deck/Porch', 'Wood', 'Composite', 'Unknown',
       'Masonry/Concrete', nan], dtype=object)

There are some `' '` values. We will replace them by NaN.

In [202]:
deck_porch_elev_labels = ['Unknown', 'No Deck/Porch', 'Composite', 'Masonry/Concrete', 'Wood', ' ']
deck_porch_elev_values = [np.nan, 0, 1, 2, 3, np.nan]
df_filter['DECKPORCHELEVATED'].replace(
    deck_porch_elev_labels,
    deck_porch_elev_values,
    inplace=True
)

#### PATIOCOVER

Text to categorical values.

In [203]:
df_filter['PATIOCOVER'].unique()

array(['Combustible', 'No Patio Cover/Carport', 'Non Combustible',
       'Unknown', nan], dtype=object)

There are some `' '` values. We will replace them by NaN.

In [204]:
patio_labels = ['Unknown', 'No Patio Cover/Carport', 'Non Combustible', 'Combustible', ' ']
patio_values = [np.nan, 0, 1, 2, np.nan]
df_filter['PATIOCOVER'].replace(
    patio_labels,
    patio_values,
    inplace=True
)

#### FENCE

Text to categorical values.

In [205]:
df_filter['FENCE'].unique()

array(['Combustible', 'Non Combustible', 'No Fence', 'Unknown', nan],
      dtype=object)

There are some `''` values. We will replace them by NaN.

In [206]:
fence_labels = ['Unknown', 'No Fence', 'Non Combustible', 'Combustible', '']
fence_values = [np.nan, 0, 1, 2, np.nan]
df_filter['FENCE'].replace(
    fence_labels,
    fence_values,
    inplace=True
)

### ROOFCONSTRUCTION

Text to integer

In [207]:
df_filter['ROOFCONSTRUCTION'].unique()

array(['Asphalt', 'Metal', 'Tile', 'Concrete', 'Other', 'Unknown', 'Wood',
       'Combustible', 'Fire Resistant', nan], dtype=object)

In [208]:
roof_construction_labels = ['Unknown', 'Fire Resistant', 'Metal', 'Concrete', 'Tile', 'Asphalt','Other','Wood', 'Combustible', ' ']
roof_construction_values = [np.nan, 0, 0, 0, 1, 2, 3 , 4, 4, np.nan]
df_filter['ROOFCONSTRUCTION'].replace(
    roof_construction_labels,
    roof_construction_values,
    inplace=True
)

In [209]:
np.sort(df_filter['ROOFCONSTRUCTION'].unique())

array([ 0.,  1.,  2.,  3.,  4., nan])

### EXTERIORSIDING

Text to int

In [210]:
df_filter['EXTERIORSIDING'].unique()

array(['Combustible', 'Ignition Resistant', 'Metal', 'Other',
       'Stucco Brick Cement', 'Vinyl', 'Wood', 'Stucco/Brick/Cement',
       'Unknown', 'Fire Resistant', nan], dtype=object)

In [211]:
exterior_siding_labels = ['Unknown', 'Metal', 'Stucco Brick Cement', 'Stucco/Brick/Cement','Ignition Resistant','Fire Resistant', 'Vinyl', 'Other','Combustible','Wood', ' ']
exterior_siding_values = [ np.nan, 0, 1, 1, 0, 0, 2, 3, 4, 4, np.nan,]
df_filter['EXTERIORSIDING'].replace(
    exterior_siding_labels,
    exterior_siding_values,
    inplace=True
)

In [212]:
np.sort(df_filter['EXTERIORSIDING'].unique())

array([ 0.,  1.,  2.,  3.,  4., nan])

### WINDOWPANE

Text to int

In [213]:
df_filter['WINDOWPANE'].unique()

array(['Multi Pane', 'Single Pane', 'No Windows', 'Unknown', nan],
      dtype=object)

In [214]:
windows_pane_labels = ['Unknown', 'No Windows', 'Single Pane', 'Multi Pane', ' ']
windows_pane_values = [np.nan, 0, 1, 2, np.nan]
df_filter['WINDOWPANE'].replace(
    windows_pane_labels,
    windows_pane_values,
    inplace=True
)

In [215]:
np.sort(df_filter['WINDOWPANE'].unique())

array([ 0.,  1.,  2., nan])

### EAVES

Text to int

In [216]:
df_filter['EAVES'].unique()

array(['Enclosed', 'No Eaves', 'Not Applicable', 'Unenclosed', 'Unknown',
       nan], dtype=object)

In [217]:
eaves_labels = ['Unknown', 'Not Applicable', 'No Eaves', 'Unenclosed', 'Enclosed', ' ']
eaves_values = [np.nan, 0, 1, 2, 3, np.nan]
df_filter['EAVES'].replace(
    eaves_labels,
    eaves_values,
    inplace=True
)

In [218]:
np.sort(df_filter['EAVES'].unique())

array([ 0.,  1.,  2.,  3., nan])

### VENTSCREEN

Text to integer

In [219]:
df_filter['VENTSCREEN'].unique()

array(['Mesh Screen <= 1/8"', 'Mesh Screen > 1/8"', 'No Vents',
       'Screened', 'Unknown', 'Unscreened', nan], dtype=object)

In [220]:
vent_screen_labels = ['Unknown', 'Screened', 'Mesh Screen <= 1/8"', 'Mesh Screen > 1/8"', 'Unscreened', 'No Vents', ' ']
vent_screen_values = [np.nan, 0, 1, 2, 3, 4, np.nan]
df_filter['VENTSCREEN'].replace(
    vent_screen_labels,
    vent_screen_values,
    inplace=True
)

In [221]:
np.sort(df_filter['VENTSCREEN'].unique())

array([ 0.,  1.,  2.,  3.,  4., nan])

### YEARBUILT

It's a number, nothing to do.

In [222]:
df_filter['YEARBUILT'].unique()

array(['1953', '1973', '1974', '1979', '1985', '1993', '1972', '1992',
       nan, '1952', '1999', '2005', '2007', '2010', '2011', '2017',
       '1960', '1988', '2013', '1900', '1940', '1942', '1943', '1946',
       '1947', '1951', '1954', '1955', '1956', '1957', '1958', '1959',
       '1961', '1962', '1963', '1964', '1967', '1968', '1969', '1970',
       '1971', '1975', '1976', '1977', '1978', '1980', '1981', '1983',
       '1986', '1987', '1989', '1990', '1991', '1994', '1995', '1996',
       '1997', '1998', '2000', '2001', '2002', '2006', '2014', '2015',
       '2016', '1930', '1944', '1949', '1950', '1965', '1966', '1982',
       '1984', '2003', '2004', '2008', '2009', '2012', '1919', '1939',
       '1945', '2019', '2018', '<Null>', '1890', '1938', '1941', '1948',
       '1877', '1915', '1910', '1929', '1937', '1922', '1914', '1880',
       '1885', '1936', '1924', '1916', '1925', '1928', '1923', '1871',
       '1913', '1934', '1920', '1935', '1905', '1931', '1933', '1906',
       

Replacing value $0$ to NaN.

In [223]:
df_filter['YEARBUILT'].replace(
    [0],
    [np.nan],
    inplace=True
)

In [224]:
df_filter.dtypes

DAMAGE               float64
ROOFCONSTRUCTION     float64
EAVES                float64
VENTSCREEN           float64
EXTERIORSIDING       float64
WINDOWPANE           float64
DECKPORCHONGRADE     float64
DECKPORCHELEVATED    float64
PATIOCOVER           float64
FENCE                float64
YEARBUILT             object
LATITUDE             float64
LONGITUDE            float64
DISTANCE             float64
dtype: object

In [225]:
df_filter.sample(10)

,DAMAGE,ROOFCONSTRUCTION,EAVES,VENTSCREEN,EXTERIORSIDING,WINDOWPANE,DECKPORCHONGRADE,DECKPORCHELEVATED,PATIOCOVER,FENCE,YEARBUILT,LATITUDE,LONGITUDE,DISTANCE
3989,2.0,2.0,3.0,2.0,4.0,1.0,1.0,0.0,0.0,0.0,1977,39.736156,-121.687984,58.609392
79583,2.0,0.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,38.488343,-122.683839,407.757636
54801,2.0,0.0,NaN,4.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,37.366214,-121.552291,102.589166
38850,2.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1956,39.754431,-121.599746,111.402965
48128,2.0,2.0,NaN,1.0,NaN,2.0,1.0,0.0,NaN,0.0,1988,38.506193,-122.612504,269.104884
39396,0.0,2.0,2.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,NaN,38.841535,-120.020011,64.493483
75164,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.101298,-122.278242,247.668606
62816,2.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,1975,34.075146,-118.925589,92.319404
5782,0.0,0.0,3.0,2.0,4.0,2.0,3.0,3.0,0.0,0.0,NaN,40.234273,-121.193497,144.546857
80381,2.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,39.296646,-123.218040,50.873155


In [ ]:
df_filter.to_pickle(DIR + 'dins_2017_2022.pkl')